In [8]:
#!pip install --upgrade langchain langchain-core langchain_community langchain_ollama langchain_text_splitters langchain_chroma streamlit
#!pip install tf-keras

In [9]:
import os
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

print("All libraries imported successfully!")

All libraries imported successfully!


In [10]:
DATA_PATH = "documents"
CHROMA_PATH = "chroma"

# FIX: Define models clearly
EMBEDDING_MODEL = "nomic-embed-text" 
LLM_MODEL = "llama3"

In [11]:
loader = DirectoryLoader(DATA_PATH, glob="**/*.pdf", loader_cls=PyPDFLoader, silent_errors=True)
documents = loader.load()

if not documents:
    print(f"No PDF documents found in '{DATA_PATH}'. Please add your notes.")
else:
    print(f"Loaded {len(documents)} document(s).")
    
    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    print(f"Split documents into {len(chunks)} chunks.")

Loaded 120 document(s).
Split documents into 337 chunks.


In [13]:
def ingest_documents():
    if not 'chunks' in globals() or not chunks:
        print("Error: No chunks found. Please run the 'Load and Split' cell first.")
        return

    print(f"Creating embeddings with '{EMBEDDING_MODEL}' and storing in Chroma... (This may take a moment)")
    
    # FIX: Use the fast embedding model
    embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)
    
    db = Chroma.from_documents(
        documents=chunks, 
        embedding=embeddings, 
        persist_directory=CHROMA_PATH
    )
    print(f"Successfully saved {len(chunks)} chunks to Chroma DB at '{CHROMA_PATH}'.")

# Run the ingestion
ingest_documents()

Creating embeddings with 'nomic-embed-text' and storing in Chroma... (This may take a moment)
Successfully saved 337 chunks to Chroma DB at 'chroma'.


In [14]:
def query_rag(question_text: str):
    print(f"Preparing to query with '{EMBEDDING_MODEL}' and '{LLM_MODEL}'...")

    # Load embeddings
    # FIX: Must use the SAME embedding model used for ingestion
    embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)
    
    # Load existing DB
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
    retriever = db.as_retriever()

    # Define prompt
    prompt_template = """
    Answer the question based only on the following context:
    {context}
    ---
    Answer the question based on the above context: {question}
    """
    prompt = ChatPromptTemplate.from_template(prompt_template)

    # Define model
    # FIX: Use the LLM for generating the final answer
    model = OllamaLLM(model=LLM_MODEL)

    # Define RAG chain
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )

    print("Querying the AI...")
    response = chain.invoke(question_text)
    return response

In [7]:
my_question = "What is Machine learning?" 
answer = query_rag(my_question)

print("\n--- AI Answer ---\n")
print(answer)

Preparing to query with 'nomic-embed-text' and 'llama3'...
Querying the AI...

--- AI Answer ---

According to the provided context, machine learning is defined as "programming computers to optimize a performance criterion using example data or past experience." It is also described as "the field of study that gives computers the ability to learn without being explicitly programmed" by Arthur Samuel in 1959.
